In [ ]:
# Please note, this code is written in R code in colab, so it is saved as .ipynb
# You can run it in colab under runtime type R or convert it into .Rmd before runing it, thanks!

library(stats)
library(ggplot2)
library(boot)

In [ ]:
# Read the dataset into R
data <- read.csv("Musk_2.csv")

# Extract features (X)
X_obs <- data[, !(names(data) %in% c("class"))]  # Exclude the column "Diagnosis"

# Function to perform Z-score normalization
z_score_normalization <- function(x) {
  return((x - mean(x)) / sd(x))
}

# Apply Z-score normalization to each column of X
X_obs <- apply(X_obs, 2, z_score_normalization)

# Extract target variable (y)
y_obs <- data$class

# Display the dimensions of X and y
# Check datatype and dimensions
cat("Datatype of X:", class(X_obs), "\n")
cat("Dimensions of X:", dim(X_obs), "\n")
cat("Datatype of y:", class(y_obs), "\n")
cat("Length of y:", length(y_obs), "\n")

Datatype of X: matrix array 
Dimensions of X: 476 166 
Datatype of y: integer 
Length of y: 476 


In [ ]:
X_obs <- as.matrix(X_obs)
X_obs <- X_obs[, 1:50]
y_obs <- as.array(y_obs)
y_obs <- array(y_obs, dim = c(dim(y_obs), 1))

cat("Datatype of X:", class(X_obs ), "\n")
cat("Dimensions of X:", dim(X_obs ), "\n")
cat("Datatype of y:", class(y_obs ), "\n")
cat("Length of y:", length(y_obs ), "\n")

Datatype of X: matrix array 
Dimensions of X: 476 50 
Datatype of y: matrix array 
Length of y: 476 


In [ ]:
# Function to calculate adjusted p-value
calculate_p_value <- function(X_obs, y_obs, sim) {

  logit_model_observed <- glm(y_obs~ ., data = as.data.frame(X_obs),
                     family = binomial(link = "logit"),
                     control = list(maxit = 1000))

  beta1_coef_obs <- coef(logit_model_observed)["f1"]
  summary_logit <- summary(logit_model_observed)
  p_values_beta1_before <- summary_logit$coefficients["f1", "Pr(>|z|)"]

  beta1_coef <- numeric(sim)
  X <- array(0, dim = c(n_max, p_max, sim))
  Y <- array(0, dim = c(n_max, sim))

  for (s in 1:sim) {
    X[, , s] <- X_obs
    X[, 15, s] <- sample(X_obs[,15], replace = TRUE)
    Y[, s] <- y_obs[, 1]
    #Y[, s] <- rbinom(n_max, 1, 0.5)

    logit_model <- glm(Y[, s] ~ ., data = as.data.frame(X[,,s]),
                     family = binomial(link = "logit"),
                     control = list(maxit = 1000))

    beta1_coef[s] <- coef(logit_model)["V1"]
  }

  p_values_beta1 <- (sum(abs(beta1_coef) >= abs(beta1_coef_obs)) + 1) / (sim + 1)

  return(list(p_values_before = p_values_beta1_before, p_values_after = p_values_beta1))
}

# Function to generate a bootstrap sample
generate_bootstrap_sample <- function(X, Y) {
  n <- nrow(X)  # Get the number of observations
  indices <- sample.int(n, replace = TRUE)
  return(list(X_boot = X[indices, , drop = FALSE], Y_boot = Y[indices, , drop = FALSE]))
}

In [ ]:
# Use boostrap to generate more X
shape_X <- dim(X_obs)
n_max <- shape_X[1]
p_max <- shape_X[2]
sim <- 30
sim_all <- 200

# Repeat the process
p_values_before_list <- vector("numeric", length = sim_all)
p_values_after_list <- vector("numeric", length = sim_all)

for (i in 1:sim_all) {
  set.seed(i)  # Setting seed for reproducibility
  boot_result <- generate_bootstrap_sample(X_obs, y_obs)
  X_obs_boot <- boot_result$X_boot
  y_obs_boot <- boot_result$Y_boot

  p_values <- calculate_p_value(X_obs_boot, y_obs_boot, sim_all)

  p_values_before_list[i] <- p_values$p_values_before
  p_values_after_list[i] <- p_values$p_values_after
}

# Plot histogram for p_values_before_list
hist(p_values_before_list, main = "P-value for Musk Dataset",
     xlab = "p-values", ylab = "Counts", col = "blue")

# Plot histogram for p_values_after_list
hist(p_values_after_list, main = "P-value for Musk Dataset with Conditional Randomization",
     xlab = "p-values", ylab = "Counts", col = "red")